In [1]:
#Import basic packages
import os
import numpy as np
import pandas as pd
import csv
import regex as re
import datetime as dt 

In [2]:
#import outcome file with org_uuid
main_df = pd.read_csv(R"d:/msc-project/data/final/outcome_final_v1.csv",encoding='utf-8',
                         parse_dates= ['founded_on','seed_date','series_a_date','series_b_date','series_c_date'])
main_processing_df = main_df[['org_uuid']]

C:\Users\Abdul Rahim Ab Ghani\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (12,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#import main organisation csv
org_ori_df = pd.read_csv(R"d:/msc-project/data/pre-processed/organisations_preprocessed.csv",encoding='utf-8',
                         parse_dates= ['founded_on','went_public_on','acquired_on','closed_on'])

In [4]:
org_ori_df.columns

Index(['org_uuid', 'name', 'legal_name', 'homepage_url', 'country_code',
       'state_code', 'region', 'city', 'address', 'postal_code', 'status',
       'short_description', 'category_list', 'category_groups_list',
       'num_funding_rounds', 'total_funding_usd', 'total_funding',
       'total_funding_currency_code', 'founded_on', 'last_funding_on',
       'closed_on', 'employee_count', 'email', 'phone', 'facebook_url',
       'linkedin_url', 'twitter_url', 'logo_url', 'primary_role', 'num_exits',
       'description', 'ipo_uuid', 'stock_exchange_symbol', 'stock_symbol',
       'went_public_on', 'share_price_usd', 'share_price',
       'share_price_currency_code', 'valuation_price_usd', 'valuation_price',
       'valuation_price_currency_code', 'money_raised_usd', 'money_raised',
       'money_raised_currency_code', 'acquisition_uuid', 'acquirer_uuid',
       'acquirer_name', 'acquirer_country_code', 'acquirer_state_code',
       'acquirer_region', 'acquirer_city', 'acquisition_type

In [5]:
org_processing_df = org_ori_df[['org_uuid','facebook_url','linkedin_url','twitter_url']]

In [6]:
org_processing_df.head(10)

,org_uuid,facebook_url,linkedin_url,twitter_url
0,e1393508-30ea-8a36-3f96-dd3226033abd,https://www.facebook.com/Wetpaint,https://www.linkedin.com/company/wetpaint,https://twitter.com/wetpainttv
1,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,http://www.facebook.com/digg,http://www.linkedin.com/company/digg,http://twitter.com/digg
2,df662812-7f97-0b43-9d3e-12f64f504fbb,https://www.facebook.com/facebook/,http://www.linkedin.com/company/facebook,https://twitter.com/facebook
3,60485007-8856-bbac-aa1b-c535c41f5f47,http://www.facebook.com/Nomadesk,NaN,http://twitter.com/Nomadesk
4,4111dc8b-c0df-2d24-ed33-30cd137b3098,NaN,NaN,http://twitter.com/geni
5,5da6106f-0d27-0d37-e9d7-dcfeccc1f709,https://www.facebook.com/twitterinc,http://www.linkedin.com/company/twitter,http://twitter.com/twitter
6,3d16cb4c-911e-75c0-de5a-15c316b39f98,https://www.facebook.com/StumbleUpon,http://www.linkedin.com/company/stumbleupon,http://twitter.com/stumbleupon
7,21e77067-5537-408e-cad7-e5e72bb6ad86,http://www.facebook.com/Scribd,http://www.linkedin.com/company/scribd,http://www.twitter.com/scribd
8,ea091a8c-40e0-0607-e05a-86e734f94ade,http://www.facebook.com/SlackerRadio,https://www.linkedin.com/company/slacker,http://twitter.com/SlackerRadio
9,59fada33-1595-de45-b362-062a04cf51cf,NaN,NaN,NaN


In [7]:
#fill NA value with 0 and non NA with 1
org_processing_df['facebook_url'][org_processing_df['facebook_url'].notnull()] = 1
org_processing_df['linkedin_url'][org_processing_df['linkedin_url'].notnull()] = 1
org_processing_df['twitter_url'][org_processing_df['twitter_url'].notnull()] = 1
org_processing_df.fillna(0,inplace=True)

C:\Users\Abdul Rahim Ab Ghani\miniconda3\lib\site-packages\pandas\core\series.py:992: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)
C:\Users\Abdul Rahim Ab Ghani\miniconda3\lib\site-packages\pandas\core\frame.py:4468: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [8]:
org_processing_df.head(10)

,org_uuid,facebook_url,linkedin_url,twitter_url
0,e1393508-30ea-8a36-3f96-dd3226033abd,1,1,1
1,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,1,1,1
2,df662812-7f97-0b43-9d3e-12f64f504fbb,1,1,1
3,60485007-8856-bbac-aa1b-c535c41f5f47,1,0,1
4,4111dc8b-c0df-2d24-ed33-30cd137b3098,0,0,1
5,5da6106f-0d27-0d37-e9d7-dcfeccc1f709,1,1,1
6,3d16cb4c-911e-75c0-de5a-15c316b39f98,1,1,1
7,21e77067-5537-408e-cad7-e5e72bb6ad86,1,1,1
8,ea091a8c-40e0-0607-e05a-86e734f94ade,1,1,1
9,59fada33-1595-de45-b362-062a04cf51cf,0,0,0


In [9]:
main_processing_df_v1 = pd.merge(main_processing_df,
                              org_processing_df,
                              on='org_uuid',
                              how='left')

In [10]:
main_processing_df_v1.shape

(38605, 4)

In [11]:
main_processing_df_v1.head(10)

,org_uuid,facebook_url,linkedin_url,twitter_url
0,eb37f7c8-c036-a915-bb7c-63f445330f66,0,0,0
1,0a88fa0e-7abe-c4ac-ab69-877b01a9cb60,0,0,0
2,de462c42-b0a5-c5af-9637-ec426b4e991f,0,0,0
3,46e0c7e4-64a3-f67e-2be2-6e707d6d5a51,0,0,0
4,89d81ccd-1b5d-4adb-1bcc-312ab6e4f90d,0,0,0
5,1b923b7c-b415-a83c-cbcc-53b27c877907,0,0,1
6,88206b5f-eefe-e06c-5a84-0dedd98dd529,0,0,0
7,91f5ca18-e026-9482-8e4f-22593943e0f5,0,1,1
8,3d607a21-39bb-651e-c53f-9890ee334f18,0,0,0
9,ab272c7d-735f-8ff7-d5b6-009ac6d91792,0,0,0


In [13]:
org_processing_df = pd.concat([org_processing_df['org_uuid'],country_one_hot],
                             axis=1)
org_processing_df.head(3)

,org_uuid,is_country_abw,is_country_afg,is_country_ago,is_country_aia,is_country_alb,is_country_are,is_country_arg,is_country_arm,is_country_asm,...,is_country_ukr,is_country_ury,is_country_usa,is_country_uzb,is_country_ven,is_country_vnm,is_country_yem,is_country_zaf,is_country_zmb,is_country_zwe
0,e1393508-30ea-8a36-3f96-dd3226033abd,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,df662812-7f97-0b43-9d3e-12f64f504fbb,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [14]:
main_processing_df_v2 = pd.merge(main_processing_df_v1,
                                org_processing_df,
                                on='org_uuid',
                                how='left')
main_processing_df_v2.shape

(38605, 186)

In [15]:
main_processing_df_v2.head(3)

,org_uuid,facebook_url,linkedin_url,twitter_url,is_country_abw,is_country_afg,is_country_ago,is_country_aia,is_country_alb,is_country_are,...,is_country_ukr,is_country_ury,is_country_usa,is_country_uzb,is_country_ven,is_country_vnm,is_country_yem,is_country_zaf,is_country_zmb,is_country_zwe
0,eb37f7c8-c036-a915-bb7c-63f445330f66,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0a88fa0e-7abe-c4ac-ab69-877b01a9cb60,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,de462c42-b0a5-c5af-9637-ec426b4e991f,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
#import event appearance file csv
event_appear_df = pd.read_csv(R"d:/msc-project/data/pre-processed/event_appearances_preprocessed.csv",encoding='utf-8')

In [17]:
event_appear_df.head(3)

,event_appear_uuid,event_appear_name,type,event_uuid,event_name,participant_uuid,participant_name,participant_type,appearance_type,short_description
0,30c4023d-b694-4704-97d7-cd99cd06ecce,Adobe MAX 2014's sponsor - CDW Corporation,event_appearance,135a927a-b238-037e-50e2-d3a2e3511ed2,Adobe MAX 2014,ae883a31-3739-7eb9-4ce0-463ee070ed79,CDW Corporation,organization,sponsor,NaN
1,dffe0542-3dac-4228-a997-3eed42cbd67e,Adobe MAX 2014's sponsor - Microsoft,event_appearance,135a927a-b238-037e-50e2-d3a2e3511ed2,Adobe MAX 2014,fd80725f-53fc-7009-9878-aeecf1e9ffbb,Microsoft,organization,sponsor,NaN
2,21884e6c-d097-4c54-8513-10536a0ea60d,Adobe MAX 2014's sponsor - Twitter,event_appearance,135a927a-b238-037e-50e2-d3a2e3511ed2,Adobe MAX 2014,5da6106f-0d27-0d37-e9d7-dcfeccc1f709,Twitter,organization,sponsor,NaN


In [18]:
#show unique count of event
event_appear_df['event_uuid'].value_counts()

44f32ad8-b8b4-4f33-9240-25ac4e64c5a3    3424
55b8f2a5-0cea-be6c-a674-a2242a6d36a4    3119
44ed4b3c-3410-a53c-ae20-bd64a63a890c    2076
746dad2c-783e-2234-f4e9-f83cdfdb28db    1627
55bf117e-47b1-4578-99e2-0f22f9d364d2    1408
                                        ... 
207c5de8-5da8-d0dc-0d85-9ecf7e19c09f       1
5d281b77-1c2c-4f49-b9d3-77741619c410       1
e5433a12-d22a-4dd1-9dfa-c85bac7c807b       1
948f6a45-e98a-4f20-b6f9-3b6e2e428cd0       1
24eed52c-e8a0-517f-b4dd-f177502dc9d3       1
Name: event_uuid, Length: 19445, dtype: int64

In [19]:
#get number of events for top 1 percentile
percentile_1 = int(0.01*19445)
top1percentile_events_uuid = event_appear_df['event_uuid'].value_counts().nlargest(percentile_1)

In [20]:
top1percentile_events_uuid

44f32ad8-b8b4-4f33-9240-25ac4e64c5a3    3424
55b8f2a5-0cea-be6c-a674-a2242a6d36a4    3119
44ed4b3c-3410-a53c-ae20-bd64a63a890c    2076
746dad2c-783e-2234-f4e9-f83cdfdb28db    1627
55bf117e-47b1-4578-99e2-0f22f9d364d2    1408
                                        ... 
608ce886-0c3c-4662-b583-b43e481dafb1     306
77efb84d-60bf-487e-963e-d5ec4360bf70     306
4cf986b2-9372-48c2-b96b-3f0a36e302a1     304
758a5b72-bde7-3250-510b-427a16c2456c     304
4bc7a540-bbfb-42ef-9331-c06325a80f6a     304
Name: event_uuid, Length: 194, dtype: int64

In [21]:
#Create df with top 1% percentile events (more than 100 occurences)
df=event_appear_df
top1percentile_events_df = df[df.groupby('event_uuid')['event_uuid'].transform('size') >= 304]

In [22]:
top1percentile_events_df

,event_appear_uuid,event_appear_name,type,event_uuid,event_name,participant_uuid,participant_name,participant_type,appearance_type,short_description
304,dd9da104-32dd-40ea-a947-73dd875504b6,CES 2015 Las Vegas's exhibitor - 3D Systems,event_appearance,9e828ece-85a3-1b8a-8634-0886c770f488,CES 2015 Las Vegas,dcf536bb-6612-908f-dbdb-4ad44e9c27bd,3D Systems,organization,exhibitor,NaN
305,6e4007b4-5f3b-42e5-b89a-d897a02f2917,CES 2015 Las Vegas's exhibitor - Analogix Semi...,event_appearance,9e828ece-85a3-1b8a-8634-0886c770f488,CES 2015 Las Vegas,e014dc5d-374c-da88-c500-7840600d8d53,Analogix Semiconductor,organization,exhibitor,NaN
306,52b71cd7-cc1a-421d-8bf3-8878a8f6c325,CES 2015 Las Vegas's exhibitor - Bosch,event_appearance,9e828ece-85a3-1b8a-8634-0886c770f488,CES 2015 Las Vegas,dcf152a1-23fc-6a0e-9a76-68c3fc2ec472,Bosch,organization,exhibitor,NaN
307,08ed3a64-d82b-4a0d-957c-01341aae5d99,CES 2015 Las Vegas's exhibitor - Intel,event_appearance,9e828ece-85a3-1b8a-8634-0886c770f488,CES 2015 Las Vegas,1e4f199c-363b-451b-a164-f94571075ee5,Intel,organization,exhibitor,NaN
7323,50f8ec76-b2bf-496b-ac5d-457dc3e52146,CES 2015 Las Vegas's exhibitor - PEOPLE PEOPLE,event_appearance,9e828ece-85a3-1b8a-8634-0886c770f488,CES 2015 Las Vegas,eba013ad-ca03-e5b1-a837-a8a067e189c2,PEOPLE PEOPLE,organization,exhibitor,NaN
...,...,...,...,...,...,...,...,...,...,...
413252,2d283427-8c75-45a0-8b8a-bf6bc50b20bc,Applied Machine Learning Days 2020's speaker -...,event_appearance,3d3c0c1c-d77b-476b-8660-ca4e505b5382,Applied Machine Learning Days 2020,b9da376c-bff2-4652-95ea-459ef17ac3a4,ICON,organization,speaker,NaN
413279,7e49a09a-d970-4f41-a71b-6e8e108a5edb,SXSW 2017's speaker - Danielle Sabrina,event_appearance,d443a495-7cc4-521b-0cc5-e436eef40614,SXSW 2017,31fb4168-39e7-41e3-a96c-be17346d5f3a,Danielle Sabrina,person,speaker,NaN
413284,63a149c1-152f-4e3f-822f-57c4823d6c7f,AWS re:Invent Las Vegas 2018's speaker - Kapil...,event_appearance,653eb0ca-b60a-43b1-9019-5914cff3e5b0,AWS re:Invent Las Vegas 2018,0a97055f-fcea-4c27-8768-900866f6c59a,Kapil Thangavelu,person,speaker,NaN
413309,da951d4c-4591-4fc4-a027-c8ac415695cc,Singapore FinTech Festival x Singapore Week of...,event_appearance,a92ee833-a276-4dee-81d9-219c3fb7c975,Singapore FinTech Festival x Singapore Week of...,f0bad717-ef4d-432c-9dca-e250b1c2ef6f,Tezos Foundation,organization,sponsor,NaN


In [23]:
top1percentile_attendees = list (set(top1percentile_events_df['participant_uuid']))
len(top1percentile_attendees)

69325

In [24]:
is_attend_top1percentile_event = list()

for org_uuid in list (main_processing_df_v2['org_uuid']):
    if org_uuid in top1percentile_attendees:
        is_attend_top1percentile_event.append(1)
    else:
        is_attend_top1percentile_event.append(0)

main_processing_df_v2['is_attend_top1percentile_event'] = is_attend_top1percentile_event

In [25]:
main_processing_df_v2.head(3)

,org_uuid,facebook_url,linkedin_url,twitter_url,is_country_abw,is_country_afg,is_country_ago,is_country_aia,is_country_alb,is_country_are,...,is_country_ury,is_country_usa,is_country_uzb,is_country_ven,is_country_vnm,is_country_yem,is_country_zaf,is_country_zmb,is_country_zwe,is_attend_top1percentile_event
0,eb37f7c8-c036-a915-bb7c-63f445330f66,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0a88fa0e-7abe-c4ac-ab69-877b01a9cb60,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,de462c42-b0a5-c5af-9637-ec426b4e991f,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
main_processing_df_v2['is_attend_top1percentile_event'].value_counts()

0    35848
1     2757
Name: is_attend_top1percentile_event, dtype: int64

In [27]:
main_processing_df_v2.head(3)

,org_uuid,facebook_url,linkedin_url,twitter_url,is_country_abw,is_country_afg,is_country_ago,is_country_aia,is_country_alb,is_country_are,...,is_country_ury,is_country_usa,is_country_uzb,is_country_ven,is_country_vnm,is_country_yem,is_country_zaf,is_country_zmb,is_country_zwe,is_attend_top1percentile_event
0,eb37f7c8-c036-a915-bb7c-63f445330f66,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0a88fa0e-7abe-c4ac-ab69-877b01a9cb60,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,de462c42-b0a5-c5af-9637-ec426b4e991f,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Organisation Description

In [37]:
org_processing_df = org_ori_df[['org_uuid','description']]

In [39]:
org_processing_df['description'][org_processing_df['description'].notnull()] = 1
org_processing_df['description'].fillna(0,inplace=True)

C:\Users\Abdul Rahim Ab Ghani\miniconda3\lib\site-packages\pandas\core\series.py:992: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)
C:\Users\Abdul Rahim Ab Ghani\miniconda3\lib\site-packages\pandas\core\series.py:4469: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [40]:
org_processing_df.head(5)

,org_uuid,description
0,e1393508-30ea-8a36-3f96-dd3226033abd,1
1,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,1
2,df662812-7f97-0b43-9d3e-12f64f504fbb,1
3,60485007-8856-bbac-aa1b-c535c41f5f47,1
4,4111dc8b-c0df-2d24-ed33-30cd137b3098,1


In [41]:
org_processing_df.rename(columns={'description':'has_description'},inplace=True)

C:\Users\Abdul Rahim Ab Ghani\miniconda3\lib\site-packages\pandas\core\frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [42]:
org_processing_df.head(3)

,org_uuid,has_description
0,e1393508-30ea-8a36-3f96-dd3226033abd,1
1,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,1
2,df662812-7f97-0b43-9d3e-12f64f504fbb,1


In [44]:
main_processing_df_v3 = pd.merge(main_processing_df_v2,
                                org_processing_df,
                                on='org_uuid',
                                how='left')

In [45]:
main_processing_df_v3.head(3)

,org_uuid,facebook_url,linkedin_url,twitter_url,is_country_abw,is_country_afg,is_country_ago,is_country_aia,is_country_alb,is_country_are,...,is_country_usa,is_country_uzb,is_country_ven,is_country_vnm,is_country_yem,is_country_zaf,is_country_zmb,is_country_zwe,is_attend_top1percentile_event,has_description
0,eb37f7c8-c036-a915-bb7c-63f445330f66,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0a88fa0e-7abe-c4ac-ab69-877b01a9cb60,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,de462c42-b0a5-c5af-9637-ec426b4e991f,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


# Rename Column and Save File

In [ ]:
main_processing_df_v2.set_index('org_uuid',inplace=True)

In [ ]:
new_column_name = list()
for column in list(main_processing_df_v2.columns):
    new_column_name.append('network_' + column)
new_column_dict = dict(zip(main_processing_df_v2.columns,new_column_name))
main_processing_df_v2.rename(columns = new_column_dict,inplace=True)
main_processing_df_v2.head(3)

In [ ]:
main_processing_df_v2.to_csv(r'd:\msc-project\data\final\features_network.csv',encoding='utf=8')